# 1. 使用モジュールとそのバージョン

In [1]:
# プログラムの実行時間測定
import time

#データ整形('mydlmodules'は自作)
import numpy as np
import pandas as pd
import mydlmodules

# 深層学習モデルの構築、学習('mydlnet'は自作)
import torch
import torch.nn as nn
import pytorch_lightning as pl
import ax
import mydlnet

# モデルの学習結果表示
import tensorboard
import tensorflow as tf

np.__version__, pd.__version__, torch.__version__, pl.__version__, ax.__version__, tensorboard.__version__, tf.__version__

('1.19.2', '1.2.4', '1.8.1+cu111', '1.3.0', '0.1.20', '2.4.1', '2.4.1')

# 2. データ取得、整形

In [2]:
# データ取得とその整形
# 'data_info.ipynb'にて詳細

time_diff_li = [i for i in range(1, 31)]

train_data = pd.read_csv('train_min_BTCJPY.csv', header=None, names=['time', 'open', 'high', 'low', 'close', 'amount'])
open_train = train_data.loc[:, 'open'].dropna().reset_index(drop=True)
open_train_f = open_train.pct_change(720).dropna().reset_index(drop=True)
pos_neg = open_train_f.mask(open_train_f >= 0, 1).mask(open_train_f < 0, 0).rename('pos_neg').astype(int)
series_f = pd.concat([open_train_f, pos_neg], axis=1)
train_loader = mydlmodules.tde_generator_class(series_f['open'], series_f['pos_neg'], time_diff_li, 300, True)

val_data = pd.read_csv('val_min_BTCJPY.csv', header=None, names=['time', 'open', 'high', 'low', 'close', 'amount'])
open_val = val_data.loc[:, 'open'].dropna().reset_index(drop=True)
open_val_f = open_val.pct_change(720).dropna().reset_index(drop=True)
pos_neg = open_val_f.mask(open_val_f >= 0, 1).mask(open_val_f < 0, 0).rename('pos_neg').astype(int)
series_f = pd.concat([open_val_f, pos_neg], axis=1)
val_loader = mydlmodules.tde_generator_class(series_f['open'], series_f['pos_neg'], time_diff_li, 50, False)

test_data = pd.read_csv('test_min_BTCJPY.csv', header=None, names=['time', 'open', 'high', 'low', 'close', 'amount'])
open_test = test_data.loc[:, 'open'].dropna().reset_index(drop=True)
open_test_f = open_test.pct_change(720).dropna().reset_index(drop=True)
pos_neg = open_test_f.mask(open_test_f >= 0, 1).mask(open_test_f < 0, 0).rename('pos_neg').astype(int)
series_f = pd.concat([open_test_f, pos_neg], axis=1)
test_loader = mydlmodules.tde_generator_class(series_f['open'], series_f['pos_neg'], time_diff_li, 50, False)

# 3. モデルの定義と学習

In [ ]:
# ハイパーパラメータ探索
# 出力結果が長い、かつ探索は何回か行っているのでコードのみ記載します。

parameters = [
    {'name': 'n_enc_layers',  'value_type':'int', 'type':'range', 'bounds':[3, 25]},
    {'name': 'n_dec_layers',  'value_type':'int', 'type':'range', 'bounds':[3, 25]}
]

def evaluation_function(parameters):
    # 下記10のパラメータが、本モデルのハイパーパラメータになります。詳細は'mydlnet.TSTransformer'のDocstringをご覧ください。
    d_val = 8
    d_at = 5
    input_len = len(time_diff_li)
    output_len = 2
    n_enc_layers = parameters.get('n_enc_layers')
    n_dec_layers = parameters.get('n_dec_layers')
    n_enc_heads = 6
    n_dec01_heads = 6
    n_dec02_heads = 6
    lr = 1e-5

    pl.seed_everything(0)
    net = mydlnet.TSTransformerClass(d_val, d_at, input_len, output_len, n_enc_layers, n_dec_layers, n_enc_heads, n_dec01_heads, n_dec02_heads, lr)
    train_logger = pl.loggers.TensorBoardLogger("tb_logs", name="ax20", default_hp_metric=False)
    checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(dirpath='check_points/ax20',
                                                                        monitor='train_loss',
                                                                        filename='{epoch:02d}-{train_loss:.2f}',
                                                                        save_last=True)

    trainer = pl.Trainer(gpus=1, max_epochs=15, benchmark=True, precision=16, amp_level='O2', logger=train_logger, callbacks=[checkpoint_callback], progress_bar_refresh_rate=0)
    trainer.fit(net, train_loader, val_loader)
    results = trainer.test(test_dataloaders=test_loader)

    val_loss = trainer.callback_metrics['val_loss']  # 検証データの損失関数を最小化できるパラメータを探索します。

    return float(val_loss)

results = ax.optimize(parameters, evaluation_function, total_trials=20, minimize=True)


In [3]:
# モデルの学習
# 探索したハイパーパラメータを用いて、モデルの学習をします。

start_time = time.time()

d_val = 8
d_at = 5
input_len = len(time_diff_li)
output_len = 2
n_enc_layers = 3
n_dec_layers = 3
n_enc_heads = 6
n_dec01_heads = 6
n_dec02_heads = 6
lr = 1e-5

pl.seed_everything(0)
net = mydlnet.TSTransformerClass(d_val, d_at, input_len, output_len, n_enc_layers, n_dec_layers, n_enc_heads, n_dec01_heads, n_dec02_heads, lr)
train_logger = pl.loggers.TensorBoardLogger("tb_logs", name="roc_log", default_hp_metric=False)
checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(dirpath='check_points/roc_model',
                                                                    monitor='train_loss',
                                                                    filename='{epoch:02d}-{train_loss:.2f}',
                                                                    save_last=True)

trainer = pl.Trainer(gpus=1, max_epochs=20, benchmark=True, precision=16, amp_level='O2', logger=train_logger, callbacks=[checkpoint_callback], progress_bar_refresh_rate=0)
trainer.fit(net, train_loader, val_loader)

results = trainer.test(test_dataloaders=test_loader)

elapsed_time = time.time() - start_time
print (str(elapsed_time) + "[sec]")

Global seed set to 0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type               | Params
----------------------------------------------------
0 | pos          | PositionalEncoding | 0     
1 | enc_layers   | ModuleList         | 4.3 K 
2 | dec_layers   | ModuleList         | 8.1 K 
3 | enc_input_fc | Linear             | 16    
4 | dec_input_fc | Linear             | 16    
5 | out_fc       | Linear             | 482   
----------------------------------------------------
12.8 K    Trainable params
0         Non-trainable params
12.8 K    Total params
0.051     Total estimated model params size (MB)
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9363871216773987, 'test_loss': 0.17385032773017883}
--------------------------------------------------------------------------------
240.2040979862213[sec]


# 4. 学習結果の表示

In [7]:
# 各データにおける精度 = 価格変化率の正負をモデルが正確に予測できた割合
trainer.callback_metrics['train_acc'], trainer.callback_metrics['val_acc'], trainer.callback_metrics['test_acc']

(tensor(0.9361, device='cuda:0'),
 tensor(0.9622, device='cuda:0'),
 tensor(0.9364, device='cuda:0'))

In [12]:
eval_transformer = mydlnet.TSTransformerClass.load_from_checkpoint('check_points/roc_model/last.ckpt',
                                                            d_val=8, d_at=5, input_len=len(time_diff_li), output_len=2,
                                                            n_enc_layers=3, n_dec_layers=3, n_enc_heads=6, n_dec01_heads=6,
                                                            n_dec02_heads=6, lr=1e-5
                                                        )

eval_transformer.eval()
eval_transformer.freeze()

batch = next(iter(test_loader))
x, t = batch
test_output = eval_transformer(x)
out_layer = nn.Softmax(dim=1)
out = out_layer(test_output)
print(torch.argmax(out, dim=1))  # モデルの出力
print(t)  # 予測値

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])
